<a href="https://colab.research.google.com/github/Mrxiba/FAFDG/blob/main/Flower_PyTorch_Federated_VNF_final_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installing dependencies

Next, we install the necessary packages for PyTorch (`torch` and `torchvision`) and Flower (`flwr`):

Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [ ]:
!pip install -q flwr[simulation] torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 7.7 MB/s eta 0:00:00


In [ ]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torch

import flwr as fl
from flwr.common import Metrics

device=DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.1.0+cu121 and Flower 1.6.0


In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

NUM_CLIENTS = 2
BATCH_SIZE = batch_size = 50
ngpu = 0
dataroot_Inline = "/content/Inline.csv"
dataroot_Passive = "/content/Passive.csv"
dataroot_vFW = "/content/vFW.csv"


class VNFDataset(Dataset):
    def __init__(self, root):
        self.df = pd.read_csv(root)
        self.data = self.df.to_numpy()
        labels=['MEM']#CPU MEM LINK
        predictors = list(set(list(self.df.columns))-set(labels))
        self.df[predictors] = self.df[predictors]/self.df[predictors].max()# guiyihua
        self.df[predictors].dropna(axis=0)
        self.df[labels] = self.df[labels]/self.df[labels].max()
        self.df[labels].dropna(axis=0)
        self.x = torch.tensor(self.df[predictors].values, dtype=torch.float32)
        self.y = torch.tensor(self.df[labels].values, dtype=torch.float32).reshape(-1, 1)
    def __getitem__(self, idx):
        return self.x[idx, :], self.y[idx,:]
    def __len__(self):
        return len(self.data)

def load_datasets():

    Inline_data = VNFDataset(dataroot_Inline)

    Passive_data = VNFDataset(dataroot_Passive)

    vFW_data = VNFDataset(dataroot_vFW)

    testset = Inline_data

    # Split training set into 2
    datasets = [vFW_data,Passive_data]

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 30  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=True))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE,shuffle=True)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets()

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(8, 24),
            nn.ReLU(),
            nn.Linear(24, 12),
            nn.ReLU(),
            nn.Linear(12, 6),
            nn.ReLU(),
            nn.Linear(6, 1)
        )

    def forward(self, x):
        x = self.linear_relu_stack(x)
        return x

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.MSELoss()
    L1 = torch.nn.L1Loss()
    optimizer = torch.optim.Adam(net.parameters()) #,lr=0.0001
    net.train()
    for epoch in range(epochs):
      epoch_loss3, epoch_loss2, epoch_loss = 0.0, 0.0, 0.0
      for images, labels in trainloader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = net(images)
        labels = labels.float()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # Metrics
        epoch_loss = loss
        loss2 = L1(outputs, labels)
        epoch_loss2 = loss2
        y_bar = outputs.mean()
        ss_tot = ((outputs-y_bar)**2).sum()
        ss_res = ((outputs-labels)**2).sum()
        epoch_loss3 = (1 - (ss_res/ss_tot)).item()
      print(f"Epoch {epoch+1}: train MSE {epoch_loss}, train MAE {epoch_loss2}, train R2 {epoch_loss3}")

def test(net, testloader):
    """Evaluate the network on the entire test set."""
    MSE_cri = torch.nn.MSELoss()
    L1 = torch.nn.L1Loss()
    RMSE_e,MAE_e,R2_e = 0.0, 0.0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
          images, labels = images.to(DEVICE), labels.to(DEVICE)
          outputs = net(images)
          labels = labels.float()
          RMSE_e = torch.sqrt(MSE_cri(outputs, labels)).item()# RMSE
          MAE_e = L1(outputs, labels).item()# MAE
          y_bar = (outputs.mean())
          ss_tot = ((labels-y_bar)**2).sum()
          ss_res = ((labels-outputs)**2).sum()
          R2_e = (1 - (ss_res/ss_tot)).item()
    return RMSE_e, MAE_e, R2_e

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]
        # Use values provided by the config
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config): ####test on local as val
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss_1, loss_2, loss_3 = test(self.net, self.valloader) #
        return float(loss_1), len(self.valloader), {"RMSE_local_val": float(loss_1),"MAE_local_val": float(loss_2),"R2_local_val": float(loss_3)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

def fit_config(server_round: int):
    """Return training configuration dict for each round.
    Perform 4 rounds of training with 30 local epoch.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning，just display.
        "local_epochs": 20,  #######
    }
    return config


# The `evaluate` function will be by Flower called after every round
def generalize(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net().to(DEVICE)
    set_parameters(net, parameters)  # Update model with the latest parameters
    RMSE, MAE, R2 = test(net, testloader) #
    print(f"Server-side unseen generalization RMSE loss {RMSE}, MAE loss{MAE}, R-Square{R2}") #
    return RMSE, {"generalization RMSE": RMSE, "generalization MAE": MAE ,"generalization R2": R2}

In [ ]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# ######Change Aggregation Strategy here<-------------Fed-Avg, xxq-FedAvg, Fed-Prox, Fed-Yogi....
strategy = fl.server.strategy.FedAvg(
    #proximal_mu =1,
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=1,  # Sample 100% of available clients for evaluation
    min_fit_clients=NUM_CLIENTS,  # Never sample less than 4 clients for training
    min_evaluate_clients=NUM_CLIENTS,  # Never sample less than 4 clients for evaluation
    min_available_clients=NUM_CLIENTS,  # Wait until all 4 clients are available
    initial_parameters=fl.common.ndarrays_to_parameters(params),## initialize parameters
    evaluate_fn=generalize,  # Pass the generalization function
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1, "num_cpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=100),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2024-01-06 12:49:48,744 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=100, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=100, round_timeout=None)
2024-01-06 12:49:53,543	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-01-06 12:49:56,447 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7698567168.0, 'object_store_memory': 3849283584.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7698567168.0, 'object_store_memory': 3849283584.0, 'CPU': 2.0}
INFO flwr 2024-01-06 12:49:56,458 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-01-06

Server-side unseen generalization RMSE loss 0.8828585147857666, MAE loss0.8730707168579102, R-Square0.0001500248908996582


(pid=12957) 2024-01-06 12:49:59.182893: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=12957) 2024-01-06 12:49:59.182971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=12957) 2024-01-06 12:49:59.184854: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=12957) 2024-01-06 12:50:01.704508: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(DefaultActor pid=12957) [Client 1] fit, config: {'server_round': 1, 'local_epochs': 20}


DEBUG flwr 2024-01-06 12:50:11,828 | server.py:236 | fit_round 1 received 2 results and 0 failures
DEBUG:flwr:fit_round 1 received 2 results and 0 failures
WARNING flwr 2024-01-06 12:50:11,838 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flwr 2024-01-06 12:50:11,875 | server.py:125 | fit progress: (1, 0.907082736492157, {'generalization RMSE': 0.907082736492157, 'generalization MAE': 0.9005072116851807, 'generalization R2': 0.0002288222312927246}, 15.16251675400008)
INFO:flwr:fit progress: (1, 0.907082736492157, {'generalization RMSE': 0.907082736492157, 'generalization MAE': 0.9005072116851807, 'generalization R2': 0.0002288222312927246}, 15.16251675400008)
DEBUG flwr 2024-01-06 12:50:11,880 | server.py:173 | evaluate_round 1: strategy sampled 2 clients (out of 2)
DEBUG:flwr:evaluate_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-01-06 12:50:11,955 | server.py:187 | evaluate_round 1 received 2 results and 0 failures
DEBUG:flwr:evaluate_round 1 received

...

Server-side unseen generalization RMSE loss 0.13056455552577972, MAE loss0.11351964622735977, R-Square2.384185791015625e-07


DEBUG flwr 2024-01-06 12:50:43,398 | server.py:236 | fit_round 100 received 2 results and 0 failures
DEBUG:flwr:fit_round 100 received 2 results and 0 failures
INFO flwr 2024-01-06 12:50:43,447 | server.py:125 | fit progress: (100, 0.12948572635650635, {'generalization RMSE': 0.12948572635650635, 'generalization MAE': 0.11269231140613556, 'generalization R2': -9.5367431640625e-07}, 46.73457822199998)
INFO:flwr:fit progress: (100, 0.12948572635650635, {'generalization RMSE': 0.12948572635650635, 'generalization MAE': 0.11269231140613556, 'generalization R2': -9.5367431640625e-07}, 46.73457822199998)
DEBUG flwr 2024-01-06 12:50:43,451 | server.py:173 | evaluate_round 100: strategy sampled 2 clients (out of 2)
DEBUG:flwr:evaluate_round 100: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-01-06 12:50:43,540 | server.py:187 | evaluate_round 100 received 2 results and 0 failures
DEBUG:flwr:evaluate_round 100 received 2 results and 0 failures
INFO flwr 2024-01-06 12:50:43,543 | server.p

Server-side unseen generalization RMSE loss 0.12948572635650635, MAE loss0.11269231140613556, R-Square-9.5367431640625e-07


History (loss, distributed):
	round 1: 0.877393364906311
	round 2: 0.8465016186237335
	round 3: 0.8224548399448395
	round 4: 0.8059016168117523
	round 5: 0.7894996404647827
	round 6: 0.7730933129787445
	round 7: 0.7566690742969513
	round 8: 0.7402818500995636
	round 9: 0.7238939702510834
	round 10: 0.7075117230415344
	round 11: 0.6911587417125702
	round 12: 0.6747960448265076
	round 13: 0.6584906876087189
	round 14: 0.6421857178211212
	round 15: 0.62591552734375
	round 16: 0.6096370220184326
	round 17: 0.5933740437030792
	round 18: 0.5771499276161194
	round 19: 0.5609356462955475
	round 20: 0.5447629988193512
	round 21: 0.528605043888092
	round 22: 0.512488454580307
	round 23: 0.4963732063770294
	round 24: 0.48029570281505585
	round 25: 0.4642877131700516
	round 26: 0.4483226537704468
	round 27: 0.4324005991220474
	round 28: 0.4165450185537338
	round 29: 0.40077444911003113
	round 30: 0.3850089907646179
	round 31: 0.36933358013629913
	round 32: 0.353758841753006
	round 33: 0.3383274823

(DefaultActor pid=12957) [Client 1] evaluate, config: {}


In [ ]:
====================================================================================end

In [ ]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# ######Change Aggregation Strategy here<-------------Fed-Avg, xxq-FedAvg, Fed-Prox, Fed-Yogi....
strategy = fl.server.strategy.FedProx(
    proximal_mu =1,
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=1,  # Sample 100% of available clients for evaluation
    min_fit_clients=2,  # Never sample less than 4 clients for training
    min_evaluate_clients=2,  # Never sample less than 4 clients for evaluation
    min_available_clients=NUM_CLIENTS,  # Wait until all 4 clients are available
    initial_parameters=fl.common.ndarrays_to_parameters(params),## initialize parameters
    evaluate_fn=generalize,  # Pass the generalization function
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1, "num_cpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=4),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-09-12 20:21:07,318 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=4, round_timeout=None)
2023-09-12 20:21:12,698	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-09-12 20:21:13,579 | app.py:210 | Flower VCE: Ray initialized with resources: {'node:10.68.107.147': 1.0, 'node:__internal_head__': 1.0, 'GPU': 1.0, 'object_store_memory': 4232332492.0, 'memory': 8464664987.0, 'CPU': 4.0, 'accelerator_type:A30': 1.0}
INFO flwr 2023-09-12 20:21:13,581 | app.py:224 | Flower VCE: Resources for each Virtual Client: {'num_gpus': 1, 'num_cpus': 1}
INFO flwr 2023-09-12 20:21:13,595 | app.py:270 | Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO flwr 2023-09-12 20:21:13,597 | server.py:89 | Initializing global parameters
INFO flwr 2023-09-12 20:21:13,598 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2023-09-12 20:21:13,599 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-09-12 20:

Server-side unseen generalization loss 0.07199699136974905 / generalization accuracy 0.11969069749277512
(DefaultActor pid=2601356) [Client 0] fit, config: {'server_round': 1, 'local_epochs': 30, 'proximal_mu': 1}
(DefaultActor pid=2601356) Epoch 1: train loss 0.02108178287744522, accuracy 0.7829212577264176
(DefaultActor pid=2601356) [Client 3] fit, config: {'server_round': 1, 'local_epochs': 30, 'proximal_mu': 1}
(DefaultActor pid=2601356) Epoch 1: train loss 0.017495084553956985, accuracy 0.8247127434304162
(DefaultActor pid=2601356) [Client 2] fit, config: {'server_round': 1, 'local_epochs': 30, 'proximal_mu': 1}
(DefaultActor pid=2601356) Epoch 1: train loss 0.02423943020403385, accuracy 0.7602030222632368
(DefaultActor pid=2601356) [Client 1] fit, config: {'server_round': 1, 'local_epochs': 30, 'proximal_mu': 1}


DEBUG flwr 2023-09-12 20:22:39,194 | server.py:236 | fit_round 1 received 4 results and 0 failures
WARNING flwr 2023-09-12 20:22:39,204 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


(DefaultActor pid=2601356) Epoch 1: train loss 0.01697658561170101, accuracy 0.8279280381838209


INFO flwr 2023-09-12 20:22:57,724 | server.py:125 | fit progress: (1, 0.03223054020349229, {'generalization accuracy': 0.6552839178317582}, 85.02661840897053)
DEBUG flwr 2023-09-12 20:22:57,725 | server.py:173 | evaluate_round 1: strategy sampled 4 clients (out of 4)


...

(DefaultActor pid=2601356) Epoch 1: train loss 0.0035067638382315636, accuracy 0.9653653163627463


INFO flwr 2023-09-12 20:27:53,297 | server.py:125 | fit progress: (4, 0.011188536295775335, {'generalization accuracy': 0.899914082636882}, 380.5993794440292)
DEBUG flwr 2023-09-12 20:27:53,299 | server.py:173 | evaluate_round 4: strategy sampled 4 clients (out of 4)


Server-side unseen generalization loss 0.011188536295775335 / generalization accuracy 0.899914082636882
(DefaultActor pid=2601356) [Client 3] evaluate, config: {}
(DefaultActor pid=2601356) [Client 2] evaluate, config: {}
(DefaultActor pid=2601356) [Client 1] evaluate, config: {}
(DefaultActor pid=2601356) [Client 0] evaluate, config: {}


DEBUG flwr 2023-09-12 20:28:00,653 | server.py:187 | evaluate_round 4 received 4 results and 0 failures
INFO flwr 2023-09-12 20:28:00,654 | server.py:153 | FL finished in 387.9569833329879
INFO flwr 2023-09-12 20:28:00,655 | app.py:225 | app_fit: losses_distributed [(1, 0.04332482980836084), (2, 0.02200467162977423), (3, 0.0193803316517415), (4, 0.018186878687130696)]
INFO flwr 2023-09-12 20:28:00,656 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-12 20:28:00,657 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-12 20:28:00,658 | app.py:228 | app_fit: losses_centralized [(0, 0.07199699136974905), (1, 0.03223054020349229), (2, 0.01671819705398189), (3, 0.010784263554537855), (4, 0.011188536295775335)]
INFO flwr 2023-09-12 20:28:00,659 | app.py:229 | app_fit: metrics_centralized {'generalization accuracy': [(0, 0.11969069749277512), (1, 0.6552839178317582), (2, 0.8486604702022963), (3, 0.8963836600796689), (4, 0.899914082636882)]}


History (loss, distributed):
	round 1: 0.04332482980836084
	round 2: 0.02200467162977423
	round 3: 0.0193803316517415
	round 4: 0.018186878687130696
History (loss, centralized):
	round 0: 0.07199699136974905
	round 1: 0.03223054020349229
	round 2: 0.01671819705398189
	round 3: 0.010784263554537855
	round 4: 0.011188536295775335
History (metrics, centralized):
{'generalization accuracy': [(0, 0.11969069749277512), (1, 0.6552839178317582), (2, 0.8486604702022963), (3, 0.8963836600796689), (4, 0.899914082636882)]}

### Behind the scenes

So how does this work? How does Flower execute this simulation?

When we call `start_simulation`, we tell Flower that there are 10 clients (`num_clients=10`). Flower then goes ahead an asks the `FedAvg` strategy to select clients. `FedAvg` knows that it should select 100% of the available clients (`fraction_fit=1.0`), so it goes ahead and selects 10 random clients (i.e., 100% of 10).

Flower then asks the selected 10 clients to train the model. When the server receives the model parameter updates from the clients, it hands those updates over to the strategy (*FedAvg*) for aggregation. The strategy aggregates those updates and returns the new global model, which then gets used in the next round of federated learning.

### Where's the accuracy?

You may have noticed that all metrics except for `losses_distributed` are empty. Where did the `{"accuracy": float(accuracy)}` go?

Flower can automatically aggregate losses returned by individual clients, but it cannot do the same for metrics in the generic metrics dictionary (the one with the `accuracy` key). Metrics dictionaries can contain very different kinds of metrics and even key/value pairs that are not metrics at all, so the framework does not (and can not) know how to handle these automatically.

As users, we need to tell the framework how to handle/aggregate these custom metrics, and we do so by passing metric aggregation functions to the strategy. The strategy will then call these functions whenever it receives fit or evaluate metrics from clients. The two possible functions are `fit_metrics_aggregation_fn` and `evaluate_metrics_aggregation_fn`.

Let's create a simple weighted averaging function to aggregate the `accuracy` metric we return from `evaluate`:

In [ ]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

The only thing left to do is to tell the strategy to call this function whenever it receives evaluation metric dictionaries from the clients:

In [ ]:
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1,
    min_fit_clients=4,
    min_evaluate_clients=4,
    min_available_clients=4,
    evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-09-10 13:50:38,118 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-09-10 13:50:43,439	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-09-10 13:50:45,114 | app.py:210 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7754735616.0, 'object_store_memory': 3877367808.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7754735616.0, 'object_store_memory': 3877367808.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-09-10 13:50:45,125 | app.py:218 | No `client_resources` specified. Using minimal resources for clients.
INFO:flwr:No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2023-09-10 13:50:45,132 | app.py:224 | Flower VCE: Resources for each Virtual Clie

History (loss, distributed):
	round 1: 0.0469954641252222
	round 2: 0.02206205269422551
	round 3: 0.019785900933225884
	round 4: 0.018084243839055655
	round 5: 0.016252841751602944
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.5494234743747302), (2, 0.7841117646651958), (3, 0.8020638801943873), (4, 0.8177082989301987), (5, 0.8371828244502929)]}

We now have a full system that performs federated training and federated evaluation. It uses the `weighted_average` function to aggregate custom evaluation metrics and calculates a single `accuracy` metric across all clients on the server side.

The other two categories of metrics (`losses_centralized` and `metrics_centralized`) are still empty because they only apply when centralized evaluation is being used. Part two of the Flower tutorial will cover centralized evaluation.

## Final remarks

Congratulations, you just trained a convolutional neural network, federated over 10 clients! With that, you understand the basics of federated learning with Flower. The same approach you've seen can be used with other machine learning frameworks (not just PyTorch) and tasks (not just CIFAR-10 images classification), for example NLP with Hugging Face Transformers or speech with SpeechBrain.

In the next notebook, we're going to cover some more advanced concepts. Want to customize your strategy? Initialize parameters on the server side? Or evaluate the aggregated model on the server side? We'll cover all this and more in the next tutorial.

## Next steps

Before you continue, make sure to join the Flower community on Slack: [Join Slack](https://flower.dev/join-slack/)

There's a dedicated `#questions` channel if you need help, but we'd also love to hear who you are in `#introductions`!

The [Flower Federated Learning Tutorial - Part 2](https://flower.dev/docs/framework/tutorial-use-a-federated-learning-strategy-pytorch.html) goes into more depth about strategies and all the advanced things you can build with them.